<a href="https://colab.research.google.com/github/QasimKhan5x/Crop-Type-Mapping/blob/main/Create_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%%shell
sudo add-apt-repository ppa:ubuntugis/ppa
sudo apt-get update
sudo apt-get install python-numpy gdal-bin libgdal-dev
pip install rasterio geopandas

In [95]:
import os
import glob
from pathlib import Path
from pprint import pprint

import rasterio as rio
from tqdm.notebook import tqdm
import numpy as np
from scipy import ndimage 
import geopandas as gpd
from osgeo import gdal_array
from matplotlib import pyplot as plt
from rasterio.plot import show, show_hist

In [4]:
grid_aoi = gpd.read_file('/content/drive/MyDrive/VarunaHackathon2022/Grid_AOI/AOI.shp')
train_aoi = gpd.read_file('/content/drive/MyDrive/VarunaHackathon2022/training_area/traindata.shp')
test_aoi = gpd.read_file('/content/drive/MyDrive/VarunaHackathon2022/testing_area/testdata.shp')

In [5]:
f2020 = '/content/drive/MyDrive/VarunaHackathon2022/sentinel-2-image/2020'
f2021 = '/content/drive/MyDrive/VarunaHackathon2022/sentinel-2-image/2021'

In [7]:
print(len(os.listdir(f2020)))
print(len(os.listdir(f2021)))

71
71


In [91]:
def resample_3d(
        stack,
        row10m,
        col10m,
        rate):
    row, col, bands = stack.shape
    # print("Array shape (%d,%d,%d)" % (row, col, bands))
    stack_10m = np.zeros((row10m, col10m, bands),dtype=np.uint16)
    # print("Resize array bands from (%d,%d,%d) to (%d,%d,%d)" % (
        # row, col, bands, row10m, col10m, bands))
    for i in range(bands):
        zoomed_arr = ndimage.zoom(stack[:, :, i], rate)
        if zoomed_arr.shape[0] > row10m:
            # reduce by 1 row and 1 column
            zoomed_arr = zoomed_arr[:-1, :-1]
        elif zoomed_arr.shape[0] < row10m:
            # pad by 1 row and 1 column
            zoomed_arr = np.pad(zoomed_arr, 1, mode='edge')[1:, 1:]
        stack_10m[:, :, i] = zoomed_arr
    del stack
    return stack_10m

In [92]:
def read_bands(img_data):
    jp2_imgs = '*.jp2'
    path = os.path.join(img_data, jp2_imgs)
    files = glob.glob(path)
    required_bands = ['B8A'] + [f'B0{i}' for i in range(2,9)] + ['B11', 'B12']
    required_bands.sort()
    req_files = [file for file in files if file[-7:-4] in required_bands]
    # 10m resolution bands
    tenM = ['02', '03', '04', '08']
    # 10M shape is (2051, 2051)
    bands = np.zeros((2051, 2051, 10))
    for i, file in enumerate(req_files):
        arr = rio.open(file).read()
        # print(f'B{file[-6:-4]}', arr.shape)
        if file[-6:-4] in tenM:
            bands[:, :, i] = arr.squeeze()
        else:
            arr = np.transpose(arr, (1, 2, 0))
            bands[:, :, i] = resample_3d(arr, 2051, 2051, 2).squeeze()
    return bands

In [93]:
path = '/content/drive/MyDrive/VarunaHackathon2022/sentinel-2-image/2020/20200102/IMG_DATA'
bands = read_bands(path)

In [96]:
def store_images(year_folder):
    '''
    Return an array of size
    71 x H X W x 10
    '''
    all_images = []
    for i, timeframe in enumerate(tqdm(os.listdir(year_folder))):
        img_data_folder = os.path.join(year_folder, timeframe, 'IMG_DATA')
        bands = read_bands(img_data_folder) 
        all_images.append(bands)
    return np.asarray(all_images)

In [ ]:
year_2020_folder = '/content/drive/MyDrive/VarunaHackathon2022/sentinel-2-image/2020'
images_2020 = store_images(year_2020_folder)

  0%|          | 0/71 [00:00<?, ?it/s]